In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from tabulate import tabulate

In [ ]:
#https://stackoverflow.com/questions/46135839/auto-detect-the-delimiter-in-a-csv-file-using-pd-read-csv
import csv

def get_delimiter(file_path, bytes = 4096):
    sniffer = csv.Sniffer()
    data = open(file_path, "r").read(bytes)
    delimiter = sniffer.sniff(data).delimiter
    return delimiter

In [ ]:
def read_csv_file(file_path):
    try:
        for encoding in ['utf-8','latin1', 'ISO-8859-1']:
            delimiter= get_delimiter(file_path)
            try:
                df = pd.read_csv(file_path, low_memory=False, encoding=encoding, delimiter=delimiter)
                return df, True, None
            except (UnicodeDecodeError, pd.errors.ParserError) as e:
                pass  
        return None, False, f"Impossible de lire {file_path}."
    except FileNotFoundError as e:
        return None, False, e

In [ ]:
years = list(range(2005, 2023, 1))
prefixes= ['caracteristiques', 'lieux', 'usagers', 'vehicules']
dataframes= []

for prefix in prefixes:
    datasets = []
    for year in years:
        connector = '_' if year <= 2016 else '-'
        file_name = f'data/raw/{prefix}{connector}{year}.csv'
        df, success, error = read_csv_file(file_name)
        if success:
            key= {file_name: df}
            datasets.append(key)
        else:
            print(f'{file_name} : {error}')
    dataframes.append(datasets)

print(f'Total datasets: {len(dataframes)}.')

for prefix, df_list in zip(prefixes, dataframes):
    print(f'{prefix}: {len(df_list)}.')

#Correction à la main des deux fichiers pour lesquels il y avait une erreur de frappe dans le nom.


In [ ]:
#recuperer le nom et le type des colonnes
def get_df_structure(df):
    return tuple(sorted((col, df[col].dtype) for col in df.columns))


def group_dataframes_by_struct(dataframes):
    #regrouper les dataframes par groupe de structures similaires
    structure_groups = {}

    for df_list in dataframes:
        for item in df_list:
            for filename, df in item.items():
                structure = get_df_structure(df)
                if structure not in structure_groups:
                    structure_groups[structure] = []
                structure_groups[structure].append(filename)
    
        #regrouper les structures
    grouped_data = []
    for i, (structure, files) in enumerate(structure_groups.items(), start=1):
        unique_dtypes = set(dtype for _, dtype in structure)
        group_info = {
            "group_number": i,
            "num_columns": len(structure),
            "unique_dtypes": len(unique_dtypes),
            "structure": structure,
            "files": files
        }
        grouped_data.append(group_info)
        
    return grouped_data

In [ ]:
# Afficher les groupes côte à côte / j'aurais pu le faire directement avec un dataframe
def display_grouped_data(grouped_data):
    table_headers = ["Groupe Numéro", "Nombre de Colonnes", "Nombre de Types Uniques", "Structure (Colonne:Type)", "Fichiers"]
    table_data = []

    for group in grouped_data:
        group_number = group['group_number']
        num_columns = group['num_columns']
        unique_dtypes = group['unique_dtypes']
        structure = group['structure']
        files = group['files']
        
        structure_str = "\n".join([f"{col}:{dtype}" for col, dtype in structure])
        files_str = "\n".join(files)
        
        row = [group_number, num_columns, unique_dtypes, structure_str, files_str]
        table_data.append(row)

    # Afficher le tableau
    print(tabulate(table_data, headers=table_headers, tablefmt="grid"))

In [ ]:
#appliquer une structure à un dataframe
def apply_structure_to_df(df, structure):
    
    struct_cols = []
    for col, _ in structure:
        struct_cols.append(col)

    for col in df.columns:
        if col not in struct_cols:
            df.drop(col, axis= 1, inplace= True)
    
    for col, dtype in structure:
        if col not in df.columns:
            df[col] = pd.Series(dtype=dtype) 
        else:
            if pd.api.types.is_integer_dtype(dtype):
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype('int64')
            elif pd.api.types.is_float_dtype(dtype):
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0.0).astype('float64')
            else:
                df[col] = df[col].astype(dtype) 
    
    df = df.reindex(columns= struct_cols)
    return df

In [ ]:
#Choisir arbitrairement les structures à garder (les plus récentes)
#j'avais fais un système de séléction automatique mais c'est une très mauvaise idée...
target_df_caract= read_csv_file('data/raw/caracteristiques-2022.csv')[0]
target_df_caract.rename(columns= {'Accident_Id':'Num_Acc'}, inplace= True)

target_df_lieux= read_csv_file('data/raw/lieux-2022.csv')[0]
target_df_usagers= read_csv_file('data/raw/usagers-2022.csv')[0]
target_df_vehicules= read_csv_file('data/raw/vehicules-2022.csv')[0]

target_df_list = [get_df_structure(target_df_caract), get_df_structure(target_df_lieux), get_df_structure(target_df_usagers), get_df_structure(target_df_vehicules)]

#remplacer
filename, df = next(iter(dataframes[0][-1].items()))
df.rename(columns= {'Accident_Id':'Num_Acc'}, inplace= True)

#brutalement 
i= 0
for df_list in dataframes:
    for item in df_list:
        for filename, df in item.items():
            df = apply_structure_to_df(df, target_df_list[i])
    i+= 1



In [ ]:
#grouper après traitement pour résultat
display_grouped_data(group_dataframes_by_struct(dataframes))

In [ ]:
#on va tenter une concaténation sur tous les fichiers
full_datasets = []

for i in range(0, 4, 1):
    full_datasets.append(pd.DataFrame()) 

i= 0
for df_list in dataframes:
    for item in df_list:
        for filename, df in item.items():
            full_datasets[i] = pd.concat([df, full_datasets[i]])
    i+= 1

In [ ]:
for df in full_datasets:
    display(df.dtypes)
    display(df.head())
    display(df.tail())

In [ ]:
#tentative pour faire un merge
merged_df = full_datasets[2]


for df_index, df in enumerate(full_datasets, start= 0):
    if df_index != 2:
        merged_df= pd.merge(merged_df, df, on= 'Num_Acc', how= 'left')

merged_df.drop_duplicates(subset= 'id_usager')
merged_df.set_index('id_usager', inplace= True)

In [ ]:
display(merged_df.head())
display(merged_df.tail())